In [1]:
import numpy as np
import time
import numba as nb
import utils
import gc
import kmeans
import correlation
import plotting
import importlib
import matplotlib.pyplot as plt
import read
import distance
import scipy.optimize as optim
import scipy.integrate as integrate
import gzip
import shutil
import os


def rho_z_integrate(z, A):
    rhol = A[0]
    rhog = A[1]
    z0 = A[2]
    d = A[3]
    rho = 0.5 * (rhol + rhog) - 0.5 * (rhol - rhog) * np.tanh((2 * (z - z0)) / d)
    return rho


def rho_z(A, z):
    rhol = A[0]
    rhog = A[1]
    z0 = A[2]
    d = A[3]
    rho = 0.5 * (rhol + rhog) - 0.5 * (rhol - rhog) * np.tanh((2 * (z - z0)) / d)
    return rho


def fit(A, z, density):
    err = np.linalg.norm(density - rho_z(A, z)) ** 2
    return err


def reload_utils():
    importlib.reload(utils)
    importlib.reload(kmeans)
    importlib.reload(correlation)
    importlib.reload(plotting)
    importlib.reload(read)
    importlib.reload(distance)

    return None


atomic_masses_trisiloxane = {
    1: 12.011,  # C
    2: 28.085,  # outer Si
    3: 15.999,  # O
    4: 1.008,  # outer H
}

mol = "trisiloxane"
AMU_TO_KG = 1.66054e-27
gc.collect()

0

- Trisiloxane 1,1,1,5,5,5 hexamethyl (3)
- Silanol trimethyl (2) 
- Cyclotrisiloxane hexamethyl (1)

In [2]:
reload_utils()
atomic_masses_trisiloxane = {
    1: 12.011,  # C
    2: 28.085,  # outer Si
    3: 15.999,  # O
    4: 1.008,  # outer H
}

mol = "trisiloxane"
temp = 400
lvc_path = f"data/LVC/{mol}/LVC_{mol}_eq{temp}K_a.dump"
with gzip.open(lvc_path + ".gz", "rb") as f_in:
    with open(lvc_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
df = read.process_data(lvc_path, atomic_masses_trisiloxane, metal_type=-1, Nt_lim=None)
lags = [12]
radii = [14]
# radii = [24]
# lags = [8]
utils.generate_dz(lags, df)
utils.generate_displacement(lags, df)
distance.generate_coord_num(radii, df)
lag = 12
rad = 14
nstart = 10
seed = 0
tol = 1e-12
cluster_vars = [f"coordination_{rad}", "pe", f"(displacement_{lag})"]
# cluster_vars = [f"coordination_{rad}", "pe"]
df["cluster_vars"] = cluster_vars
pe = np.sum(df["molecule"]["pe"], axis=1)
_, start, _ = correlation.actime(pe)
molecule_phase, atom_phase, centroids, err = kmeans.classify_phase(
    df, cluster_vars, start=start
)
percent_gas_kmeans = np.mean(df["Ngas"][lag + start :] / df["Nmolecule"])
percent_liq_kmeans = np.mean(df["Nliquid"][lag + start :] / df["Nmolecule"])



Reading Data...



100%|██████████| 1001/1001 [02:32<00:00,  6.57it/s]



Converting to molecule...Done
Generating Coordination Number...Done


In [5]:
del df
gc.collect()
try:
    os.remove(lvc_path)
except Exception as e:
    print(e)

In [ ]:
temps = [x for x in range(225, 625, 25) if x != 550]

rho_cs = np.zeros(len(temps))
rho_gs_actual = np.zeros(len(temps))
rho_gs_kmeans = np.zeros(len(temps))
rho_ls_actual = np.zeros(len(temps))
rho_ls_kmeans = np.zeros(len(temps))
percents_liq_actual = np.zeros(len(temps))
percents_gas_actual = np.zeros(len(temps))
percents_liq_kmeans = np.zeros(len(temps))
percents_gas_kmeans = np.zeros(len(temps))

for i, temp in enumerate(temps):
    lvc_path = f"data/LVC/{mol}/LVC_{mol}_eq{temp}K_a.dump"

    with gzip.open(lvc_path + ".gz", "rb") as f_in:
        with open(lvc_path, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
            df = read.process_data(
                lvc_path, atomic_masses_trisiloxane, metal_type=-1, Nt_lim=None
            )
            radii = [14]
            lags = [12]
            utils.generate_dz(lags, df)
            utils.generate_displacement(lags, df)
            distance.generate_coord_num(radii, df)

            lag = 12
            rad = 14
            nstart = 10
            seed = 0
            tol = 1e-12
            cluster_vars = [f"coordination_{rad}", "pe", f"(displacement_{lag})"]
            # cluster_vars = [f"log(displacement_{lag})"]
            pe = np.sum(df["molecule"]["pe"], axis=1)
            df["cluster_vars"] = cluster_vars
            _, start, _ = correlation.actime(pe)
            molecule_phase, atom_phase, _, err = kmeans.classify_phase(
                df, cluster_vars, start=start, external_centroids=centroids
            )
            percent_gas_kmeans = np.mean(df["Ngas"][lag + start :] / df["Nmolecule"])
            percent_liq_kmeans = np.mean(df["Nliquid"][lag + start :] / df["Nmolecule"])

            z = df["molecule"]["z"]
            L = utils.get_L(df)
            com = np.mean(z, axis=1)
            z = z - com[:, np.newaxis]
            zmin = np.max(np.min(z, axis=1))  # lowest z bin present for all t
            zmax = np.min(np.max(z, axis=1))  # highest z bin present for all t
            hist_range = (zmin, zmax)
            nbins = 200
            density, zbin = utils.density(z, nbins, hist_range, time_avg=True)
            density /= df["Nmolecule"]

            try:
                A0 = np.array([0.001, 0.0005, 60, 20])  # rho_l, rho_g, z0, d
                train_z = np.abs(zbin).copy()
                xopt = optim.fmin(func=fit, x0=A0, args=(train_z, density), disp=False)
                rho_l, rho_g, z0, d = xopt

                rho = rho_z(xopt, train_z)
                rho_coexist = rho[np.argmin(np.abs(train_z - z0))]
                s = np.argsort(train_z)

                percent_liq, err_liq = integrate.quad(
                    rho_z_integrate, 0, z0, args=(xopt), full_output=0
                )
                percent_gas, err_gas = integrate.quad(
                    rho_z_integrate, z0, zmax, args=(xopt), full_output=0
                )
                total = percent_gas + percent_liq
                percent_gas /= total
                percent_liq /= total

                fig, ax = plt.subplots(figsize=(8, 6), dpi=200)
                dz = np.abs(zbin[1] - zbin[0])
                slice_volume = dz * L[0] * L[1] * 1e-10**3
                mol_weight = df["molecule"]["mass"] * AMU_TO_KG
                norm = df["Nmolecule"] * mol_weight / slice_volume
                mask_g = molecule_phase == 1
                mask_l = molecule_phase == 0
                z_g = z[:, mask_g]
                z_l = z[:, mask_l]
                z_g_min = np.max(np.min(z_g, axis=1))
                z_g_max = np.min(np.max(z_g, axis=1))
                z_l_min = np.max(np.min(z_l, axis=1))
                z_l_max = np.min(np.max(z_l, axis=1))
                hist_range_g = (z_g_min, z_g_max)
                hist_range_l = (z_l_min, z_l_max)
                density_g, zbin_g = utils.density(
                    z_g, nbins, hist_range_g, time_avg=True
                )
                density_l, zbin_l = utils.density(
                    z_l, nbins, hist_range_l, time_avg=True
                )
                density_g /= df["Ngas"]
                density_l /= df["Nliquid"]
                density_g *= norm
                density_l *= norm
                rho_gs_kmeans[i] = np.mean(density_g)
                rho_ls_kmeans[i] = np.mean(density_l)
                rho_gs_actual[i] = rho_g * norm
                rho_ls_actual[i] = rho_l * norm

                y1 = density * norm
                y2 = rho * norm
                x = zbin
                ylim = (0, 1.1 * max(y1.max(), y2.max()))

                ax.set_xlabel("Z (Å)")
                ax.set_ylabel(r"$\rho \ (kg \cdot m^{-3})$")
                ax.set_title(f"{mol} {temp}K Density Profile")
                ax.set_xlim(zmin, zmax)
                ax.set_ylim(ylim)
                ax.plot(x, y1, label="data")
                ax.plot(x, y2, label="fit")
                ax.vlines(
                    z0,
                    0,
                    ylim[1],
                    color="tab:red",
                    linestyle="--",
                    label="z0",
                    zorder=1,
                )
                # ax.vlines(-z0, 0, ylim[1], color="tab:red", linestyle="--", zorder=0)
                ax.hlines(
                    rho_l * norm,
                    zmin,
                    0,
                    linestyle=(0, (5, 10)),
                    color="black",
                    zorder=0,
                )
                ax.text(zmin * 0.7, rho_l * norm * 1.02, r"$\rho_l$", fontsize=12)
                ax.text(zmin * 0.7, rho_g * norm * 1.7, r"$\rho_g$", fontsize=12)

                start = z0 - d / 2
                end = z0 + d / 2
                start_arg = np.argmin(np.abs(zbin - start))
                end_arg = np.argmin(np.abs(zbin - end))
                start_y = rho[start_arg] * norm
                end_y = rho[end_arg] * norm
                ax.vlines(
                    start,
                    0,
                    start_y * 0.95,
                    color="black",
                    linestyle="--",
                    label="d",
                    zorder=0,
                )
                ax.vlines(end, 0, end_y * 0.95, color="black", linestyle="--", zorder=0)

                gas_start = np.argmin(np.abs(zbin - z0))
                gas_end = np.argmin(np.abs(zbin + z0))

                liq_start = gas_end
                liq_end = gas_start

                ax.fill_between(
                    x[gas_start:],
                    y2[gas_start:],
                    color="tab:red",
                    alpha=0.3,
                    label="gas",
                )
                ax.fill_between(
                    x[:gas_end],
                    y2[:gas_end],
                    color="tab:red",
                    alpha=0.3,
                    zorder=0,
                )

                ax.fill_between(
                    x[liq_start - 1 : liq_end + 1],
                    y2[liq_start - 1 : liq_end + 1],
                    color="tab:blue",
                    alpha=0.3,
                    label="cond.",
                    zorder=0,
                )

                ax.text(-10, 0.5 * ylim[1], f"{percent_liq * 100:.1f}%", fontsize=12)
                ax.text(
                    -10,
                    0.45 * ylim[1],
                    f"{percent_liq_kmeans * 100:.1f}%",
                    fontsize=12,
                    color="tab:green",
                )
                ax.text(
                    0.6 * zmax,
                    1.5 * rho_g * norm,
                    f"{percent_gas * 100:.1f}%",
                    fontsize=12,
                )
                ax.text(
                    0.75 * zmax,
                    1.5 * rho_g * norm,
                    f"{percent_gas_kmeans * 100:.1f}%",
                    fontsize=12,
                    color="tab:green",
                )

                x = z0
                y = rho_coexist * norm
                rho_cs[i] = y
                percents_liq_actual[i] = percent_liq
                percents_gas_actual[i] = percent_gas
                percents_liq_kmeans[i] = percent_liq_kmeans
                percents_gas_kmeans[i] = percent_gas_kmeans

                ax.scatter(x, y, color="black", zorder=10, s=15)
                ax.text(x * 1.1, y, rf"$\rho_c$ = {y:.1f}", fontsize=10)

                ax.legend()

                fig.savefig(f"data/LVC/{mol}/Figures/{mol}_{temp}K_density_profile.png")

                plt.close()

            except Exception as e:
                print(e)

    del df
    try:
        os.remove(lvc_path)
    except Exception as e:
        print(e)

    gc.collect()

mask = rho_cs > 0
temps = np.array(temps)[mask]
rho_cs = rho_cs[mask]
fig, ax = plt.subplots(figsize=(8, 6), dpi=200)
ax.set_xlabel(r"$\rho \ (kg \cdot m^{-3})$")
ax.set_ylabel("T (K)")
ax.set_title(f"{mol} LVC")
ax.plot(rho_cs, temps, marker="o", color="tab:blue")
fig.savefig(f"data/LVC/{mol}/Figures/{mol}_LVC.png")
plt.show()


Reading Data...



100%|██████████| 1001/1001 [02:22<00:00,  7.05it/s]



Converting to molecule...Done
Generating Coordination Number...Done
too many indices for array: array is 2-dimensional, but 3 were indexed

Reading Data...



In [ ]:
y = np.array(temps)[5:9]
x = rho_cs[5:9]
fig, ax = plt.subplots(figsize=(8, 6), dpi=200)
ax.set_xlabel(r"$\rho \ (kg \cdot m^{-3})$")
ax.set_ylabel("T (K)")
ax.set_title(f"{mol} LVC")
ax.plot(x, y, marker="o", color="tab:blue")
fig.savefig(f"data/LVC/{mol}/Figures/{mol}_LVC.png")
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=200)
ax.set_xlabel("T (K)")
ax.set_ylabel("Percent Liquid")
ax.set_title(f"{mol} Percent Liquid")
ax.plot(
    temps[5:9],
    percents_liq_actual[5:9] * 100,
    marker="o",
    color="tab:blue",
    label="LVC",
)
ax.plot(
    temps, percents_liq_kmeans * 100, marker="o", color="tab:green", label="K-means"
)
ax.legend()
fig.savefig(f"data/LVC/{mol}/Figures/{mol}_percent_liquid.png")
plt.show()

fig, ax = plt.subplots(figsize=(8, 6), dpi=200)
ax.set_xlabel("T (K)")
ax.set_ylabel("Percent Gas")
ax.set_title(f"{mol} Percent Gas")
ax.plot(
    temps[5:9],
    percents_gas_actual[5:9] * 100,
    marker="o",
    color="tab:blue",
    label="LVC",
)
ax.plot(
    temps, percents_gas_kmeans * 100, marker="o", color="tab:green", label="K-means"
)
ax.legend()
fig.savefig(f"data/LVC/{mol}/Figures/{mol}_percent_gas.png")
plt.show()

In [ ]:
reload_utils()
atomic_masses_trisiloxane = {
    1: 12.011,  # C
    2: 28.085,  # outer Si
    3: 15.999,  # O
    4: 1.008,  # outer H
}

mol = "trisiloxane"
temp = 400
lvc_path = f"data/LVC/{mol}/LVC_{mol}_eq{temp}K_a.dump"
with gzip.open(lvc_path + ".gz", "rb") as f_in:
    with open(lvc_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
df = read.process_data(lvc_path, atomic_masses_trisiloxane, metal_type=-1, Nt_lim=None)


In [ ]:
lags = [i for i in range(6, 14, 2)]
radii = [i for i in range(14, 28, 2)]
# radii = [24]
# lags = [8]
utils.generate_dz(lags, df)
utils.generate_displacement(lags, df)
distance.generate_coord_num(radii, df)


In [3]:
temp = 250
lvc_path = f"data/LVC/{mol}/LVC_{mol}_eq{temp}K_a.dump"
# with gzip.open(lvc_path + ".gz", "rb") as f_in:
#     with open(lvc_path, "wb") as f_out:
#         shutil.copyfileobj(f_in, f_out)
df250 = read.process_data(
    lvc_path, atomic_masses_trisiloxane, metal_type=-1, Nt_lim=None
)


Reading Data...



100%|██████████| 1001/1001 [02:39<00:00,  6.26it/s]



Converting to molecule...Done


In [4]:
lags = [i for i in range(6, 14, 2)]
radii = [i for i in range(14, 28, 2)]
lags = [12]
radii = [14]
# radii = [24]
# lags = [8]
utils.generate_dz(lags, df250)
utils.generate_displacement(lags, df250)
distance.generate_coord_num(radii, df250)

Generating Coordination Number...Done


In [ ]:
# from sklearn.mixture import GaussianMixture

# cluster_vars = [f"coordination_{rad}", "pe", f"log(dz_{lag})"]
# data, Nt, max_lag = kmeans.prep_data(df, cluster_vars)
# gm = GaussianMixture(n_components=2, random_state=0).fit(data)
# phase = gm.predict(data)
# print(np.sum(phase == 0) / (df["Nt"] - max_lag) / df["Nmolecule"])
# print(np.sum(phase == 1) / (df["Nt"] - max_lag) / df["Nmolecule"])

In [5]:
lag = 12
rad = 14
nstart = 10
seed = 0
tol = 1e-12
cluster_vars = [f"coordination_{rad}", "pe", f"(displacement_{lag})"]
# cluster_vars = [f"coordination_{rad}", "pe"]
df["cluster_vars"] = cluster_vars
pe = np.sum(df["molecule"]["pe"], axis=1)
_, start, _ = correlation.actime(pe)
molecule_phase, atom_phase, centroids, err = kmeans.classify_phase(
    df, cluster_vars, start=start
)
percent_gas_kmeans = np.mean(df["Ngas"][lag + start :] / df["Nmolecule"])
percent_liq_kmeans = np.mean(df["Nliquid"][lag + start :] / df["Nmolecule"])

In [ ]:
print(percent_gas_kmeans)

In [6]:
df250["cluster_vars"] = cluster_vars
pe = np.sum(df250["molecule"]["pe"], axis=1)
_, start, _ = correlation.actime(pe)
molecule_phase, atom_phase, centroids250, err = kmeans.classify_phase(
    df250, cluster_vars, start=start, external_centroids=centroids
)
percent_gas_kmeans250 = np.mean(df250["Ngas"][lag + start :] / df250["Nmolecule"])
percent_liq_kmeans250 = np.mean(df250["Nliquid"][lag + start :] / df250["Nmolecule"])


each timestep is a millionth of a nanosecond, or a femtosecond. So if dt = 10000, then each step is 10 picoseconds which is one one hundreth of a nanosecond. If dt = 50000, then each step is 50 picoseconds, which is 5 hundredths of a nanosecond 

In [8]:
reload_utils()
plotting.scatter_var_step(
    df250,
    "phase",
    250,
    mode="mol",
    traces=True,
    ntraces=5,
    reversescale=None,
    metal=False,
    metal_dynamics=False,
    top=False,
    bottom=False,
)


In [281]:
zmax = df["bounds"][-1, -1]
zmin = df["bounds"][-1, 0]
z = df["molecule"]["z"]
com = np.mean(z, axis=1)
# print(com.shape)
z = z - com[:, np.newaxis]
hist_range = (zmin, zmax)
nbins = 200
density, zbin = utils.density(z, nbins, hist_range, time_avg=False)
density /= df["Nmolecule"]

In [ ]:
plt.plot(com)
plt.xlabel("t")
plt.ylabel("z com")
plt.show()

In [ ]:
# animate density at each t

from matplotlib import animation
from IPython.core.display import HTML

fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xlabel("z")
ax.set_ylabel("rho")
ax.set_title("Density profile")
ax.set_xlim(zmin, zmax)
ylim = (0, density.max() * 100)
ax.set_ylim(ylim)
plt.close()


def animate(t):
    ax.clear()
    z_plot = np.linspace(zmin, zmax, num=nbins, endpoint=True)
    ax.plot(z_plot[1:-1], density[t][1:-1] * 100)
    ax.set_xlabel("z")
    ax.set_ylabel("Molecule %")
    ax.set_title(f"Molecular Density Profile t = {t}")
    ax.set_xlim(zmin, zmax)
    ax.set_ylim(ylim)


frames = np.arange(0, df["Nt"], 10)
anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100)
HTML(anim.to_html5_video())

In [ ]:
reload_utils()
plotting.plot_density(df, nbins=nbins, norm="percent")
plt.show()

In [ ]:
z = df["molecule"]["z"]
L = utils.get_L(df)
com = np.mean(z, axis=1)
z = z - com[:, np.newaxis]
zmin = np.max(np.min(z, axis=1))  # lowest z bin present for all t
zmax = np.min(np.max(z, axis=1))  # highest z bin present for all t
print(zmin, zmax)

In [287]:
hist_range = (zmin, zmax)
nbins = 200
density, zbin = utils.density(z, nbins, hist_range, time_avg=True)
density /= df["Nmolecule"]

In [ ]:
A0 = np.array([0.001, 0.0005, 60, 20])  # rho_l, rho_g, z0, d
train_z = np.abs(zbin).copy()
xopt = optim.fmin(func=fit, x0=A0, args=(train_z, density))
rho_l, rho_g, z0, d = xopt

print(
    f"\nrho_l: {rho_l * 100:.4f}%\nrho_g: {rho_g * 100:.4f}%\nz0: {z0:.4f}Å\nd: {d:.4f}Å"
)
rho = rho_z(xopt, train_z)
rho_coexist = rho[np.argmin(np.abs(train_z - z0))]
s = np.argsort(train_z)

In [ ]:
percent_liq, err_liq = integrate.quad(
    rho_z_integrate, 0, z0, args=(xopt), full_output=0
)
percent_gas, err_gas = integrate.quad(
    rho_z_integrate, z0, zmax, args=(xopt), full_output=0
)
total = percent_gas + percent_liq
percent_gas /= total
percent_liq /= total
print(
    f"percent gas: {percent_gas * 100:.4f}%\npercent liquid: {percent_liq * 100:.4f}%"
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=200)
dz = np.abs(zbin[1] - zbin[0])
slice_volume = dz * L[0] * L[1] * 1e-10**3
mol_weight = df["molecule"]["mass"] * AMU_TO_KG
norm = df["Nmolecule"] * mol_weight / slice_volume

y1 = density * norm
y2 = rho * norm
x = zbin
ylim = (0, 1.1 * max(y1.max(), y2.max()))

ax.set_xlabel("Z (Å)")
ax.set_ylabel(r"$\rho \ (kg \cdot m^{-3})$")
ax.set_title(f"{mol} {temp}K Density Profile")
ax.set_xlim(zmin, zmax)
ax.set_ylim(ylim)
ax.plot(x, y1, label="data")
ax.plot(x, y2, label="fit")
ax.vlines(z0, 0, ylim[1], color="tab:red", linestyle="--", label="z0", zorder=1)
# ax.vlines(-z0, 0, ylim[1], color="tab:red", linestyle="--", zorder=0)
ax.hlines(rho_l * norm, zmin, 0, linestyle=(0, (5, 10)), color="black", zorder=0)
ax.text(zmin * 0.7, rho_l * norm * 1.02, r"$\rho_l$", fontsize=12)
ax.text(zmin * 0.7, rho_g * norm * 1.7, r"$\rho_g$", fontsize=12)

start = z0 - d / 2
end = z0 + d / 2
start_arg = np.argmin(np.abs(zbin - start))
end_arg = np.argmin(np.abs(zbin - end))
start_y = rho[start_arg] * norm
end_y = rho[end_arg] * norm
ax.vlines(start, 0, start_y * 0.95, color="black", linestyle="--", label="d", zorder=0)
ax.vlines(end, 0, end_y * 0.95, color="black", linestyle="--", zorder=0)

gas_start = np.argmin(np.abs(zbin - z0))
gas_end = np.argmin(np.abs(zbin + z0))

liq_start = gas_end
liq_end = gas_start

ax.fill_between(x[gas_start:], y2[gas_start:], color="tab:red", alpha=0.3, label="gas")
ax.fill_between(
    x[:gas_end],
    y2[:gas_end],
    color="tab:red",
    alpha=0.3,
    zorder=0,
)

ax.fill_between(
    x[liq_start - 1 : liq_end + 1],
    y2[liq_start - 1 : liq_end + 1],
    color="tab:blue",
    alpha=0.3,
    label="cond.",
    zorder=0,
)


ax.text(-10, 0.5 * ylim[1], f"{percent_liq * 100:.1f}%", fontsize=12)
ax.text(
    -10,
    0.45 * ylim[1],
    f"{percent_liq_kmeans * 100:.1f}%",
    fontsize=12,
    color="tab:green",
)
ax.text(0.6 * zmax, 1.5 * rho_g * norm, f"{percent_gas * 100:.1f}%", fontsize=12)
ax.text(
    0.75 * zmax,
    1.5 * rho_g * norm,
    f"{percent_gas_kmeans * 100:.1f}%",
    fontsize=12,
    color="tab:green",
)

x = z0
y = rho_coexist * norm

ax.scatter(x, y, color="black", zorder=10, s=15)
ax.text(x * 1.1, y, rf"$\rho_c$ = {y:.1f}", fontsize=10)

ax.legend()

plt.show()

In [ ]:
np.mean(df["Ngas"][lag:] / df["Nmolecule"])


In [ ]:
plt.plot(df["Nliquid"][lag:] / df["Nmolecule"], color="tab:blue")
# plt.plot(df["Ngas"][lag:] / df["Nmolecule"], color="tab:orange")
plt.show()

In [ ]:
print(np.sum(molecule_phase == 0, axis=1))

In [ ]:
n_gas = np.sum(molecule_phase == 0, axis=0)
n_gas, df["Ngas"].shape, n_gas.shape

In [15]:
reload_utils()
v = utils.get_v(df)
vacf = correlation.calc_vacf0(v)

In [ ]:
plt.plot(vacf[10:])

In [ ]:
df["dt"] / 1e15

In [17]:
reload_utils()
dt = 1e-12
dt = 5e-11
D = correlation.diffusion_constant(vacf, dt)


In [ ]:
D

In [3]:
L = utils.get_L(df)
R = utils.get_pos(df)
distance = utils.distance(R, L)

In [8]:
dr = 0.5
g, r = correlation.pair_correlation(distance, dr, L)

In [ ]:
df["molecule"]["mass"]

In [31]:
values = df["atom"]["timestep"]
Nt = df["Nt"]
Natom_per_molecule = df["Natom_per_molecule"]
Nmolecule = df["Nmolecule"]
avg = utils.molecule_average(values, Nt, Nmolecule, Natom_per_molecule).astype(np.int32)


In [ ]:
x = df["atom"]["timetep"]
np.sum(df["atom"]["timestep"][:, 0])

In [ ]:
df["molecule"]["timestep"][:, 0][-30:]

In [ ]:
np.int32(50000000)

In [ ]:
df["timesteps"]

In [ ]:
np.unique(df["atom"]["timestep"][:, 0])

In [ ]:
df["timesteps"].max()

In [ ]:
np.int16(df["timesteps"].max())

In [ ]:
np.sort(g)[:10]

In [ ]:
plotting.plot_pair_correlation(g, r)
plt.show()

In [ ]:
n = np.arange(1, 10)
n2 = np.arange(10, 100, 10)
n = np.concatenate((n, n2))
kvecs, kmags, kmask, start = sk_init(df, n)
r = get_pos(df).T
mean_sk = sk_time_average(r, kvecs, kmask, start)


In [ ]:
bin_centers, sk_bin = sk_binned(df, n, mean_sk, dw=1)
fig, ax = plt.subplots()
ax.plot(bin_centers, sk_bin)
ax.scatter(bin_centers, sk_bin, s=10)
plt.show()


In [ ]:
reload_utils()
lags = [i for i in range(6, 10, 2)]
radii = [i for i in range(14, 28, 2)]
utils.generate_dz(lags, df)
utils.generate_coord_num(radii, df)
utils.generate_displacement(lags, df)


In [ ]:
metal_mask = df["metal_mask"]
pe_mol = np.sum(df["molecule"]["pe"], axis=1)
ke_mol = np.sum(df["molecule"]["ke"], axis=1)
pe_metal = np.sum(df["atom"]["pe"][:, metal_mask], axis=1)
ke_metal = np.sum(df["atom"]["ke"][:, metal_mask], axis=1)
E_total = pe_mol + ke_mol + pe_metal + ke_metal
E_mol = pe_mol + ke_mol


In [ ]:
%matplotlib inline
fig1, ax1 = utils.plot_energyac(
    E_total, ylabel="Total System Energy", title="Trisiloxane 500K"
)
fig2, ax2 = utils.plot_energyac(
    E_mol, ylabel="Total Molecule Energy", title="Trisiloxane 500K"
)
fig3, ax3 = utils.plot_energyac(
    pe_metal + pe_mol, ylabel="Total Potential Energy", title="Trisiloxane 500K"
)
fig4, ax4 = utils.plot_energyac(
    ke_metal + ke_mol, ylabel="Total Kinetic Energy", title="Trisiloxane 500K"
)
plt.close("all")
# plt.show()


In [ ]:
reload_utils()
lag = 8
rad = 20
nstart = 10
seed = 0
tol = 1e-12
t = 80
# lag = 0
cluster_vars = [f"coordination_{rad}", "pe", f"log(dz_{lag})"]
cluster_vars = [f"log(displacement_{lag})"]
# cluster_vars = [f"coordination_26"]
df["cluster_vars"] = cluster_vars
molecule_phase, atom_phase, centroids, err = kmeans.classify_phase(
    df, cluster_vars, start=400
)


In [ ]:
plot_rate(df, title="Evaporation Rate - Condensation Rate")
